In [ ]:
from pathlib import Path
import os
import sys
import shutil
from datetime import date, timedelta
import xarray as xr
import netCDF4 as nc
import numpy as np

In [ ]:
IDX_MIN_LON = 796 
IDX_MIN_LAT = 645 

IMG_SIZE_LON = 180
IMG_SIZE_LAT = 180

TIME_LIST = [str(i).zfill(2) for i in range(24)] 

In [ ]:
PATH_URMA_ORIGINAL = "/data1/ai-datadepot/models/urma/2p5km/grib2"
PATH_URMA_TRAIN = "/scratch/RTMA/alex.schein/URMA_train_test/train"
PATH_URMA_TEST = "/scratch/RTMA/alex.schein/URMA_train_test/test"

In [ ]:
START_DATE_TRAIN = date(2021,1,1) #should be jan 1, 2021
END_DATE_TRAIN = date(2023,12,31) #should be dec 31, 2023
NUM_DAYS_TRAIN = END_DATE_TRAIN-START_DATE_TRAIN

START_DATE_TEST = date(2024,1,1) #should be jan 1, 2024
END_DATE_TEST = date(2024,12,31) #should be dec 31, 2024
NUM_DAYS_TEST = END_DATE_TEST-START_DATE_TEST

In [ ]:
# #### DIAGNOSE MISSING FILES

## As of 6/23, no files are missing

# for i in range(NUM_DAYS_TRAIN.days + 1):
#     DATE_STR_TRAIN = date.strftime(START_DATE_TRAIN + timedelta(days=i), "%Y%m%d")
#     filenames = os.listdir(PATH_URMA_ORIGINAL+"/"+DATE_STR_TRAIN)
#     for time in TIME_LIST:
#         try:
#             filename = [x for x in filenames if time in x and ".idx" not in x][0] #will only be one matching filename @ appropriate time
#         except:
#             print(f"{DATE_STR_TRAIN} | {time}z is missing")

# for i in range(NUM_DAYS_TEST.days + 1):
#     DATE_STR_TEST = date.strftime(START_DATE_TEST + timedelta(days=i), "%Y%m%d")
#     filenames = os.listdir(PATH_URMA_ORIGINAL+"/"+DATE_STR_TEST)
#     for time in TIME_LIST:
#         try:
#             filename = [x for x in filenames if time in x and ".idx" not in x][0] #will only be one matching filename @ appropriate time
#         except:
#             print(f"{DATE_STR_TEST} | {time}z is missing")

In [ ]:
# All 2021-2023 files to PATH_TRAIN directory
for i in range(NUM_DAYS_TRAIN.days + 1):
    DATE_STR_TRAIN = date.strftime(START_DATE_TRAIN + timedelta(days=i), "%Y%m%d")
    filenames = os.listdir(PATH_URMA_ORIGINAL+"/"+DATE_STR_TRAIN)
    # temp_filenames = os.listdir(PATH_URMA_TEMP)
    for time in TIME_LIST:
        try: #missing some 2021/04/19 files, as of 5/23
            filename = [x for x in filenames if time in x and ".idx" not in x][0] #will only be one matching filename @ appropriate time
            new_filename = f"urma_{DATE_STR_TRAIN}_t{time}z.nc"
            if not os.path.exists(PATH_URMA_TRAIN+f"/{new_filename}"):
                #throws a hissy fit as it can't write an index file on ai-datadepot, but it should still compute fine...
                t2m = xr.open_dataset(PATH_URMA_ORIGINAL+"/"+DATE_STR_TRAIN+"/"+filename, engine='cfgrib', decode_timedelta=True)
                t2m = t2m.t2m
                t2m_subset = t2m.isel(y=slice(IDX_MIN_LAT, IDX_MIN_LAT+IMG_SIZE_LAT),
                                      x=slice(IDX_MIN_LON, IDX_MIN_LON+IMG_SIZE_LON))
                t2m_subset.to_netcdf(PATH_URMA_TRAIN+"/"+new_filename)#, encoding={"t2m":{"zlib":True, "complevel":9}}) #(6/16) DON'T USE ENCODING - though it saves space, it automatically enables chunking which is SUPER slow!
                print(f"{new_filename} written to {PATH_URMA_TRAIN}")
        except: #This is a hack workaround for those missing 2021/04/19 files and should be removed once those are done
            print(f"{DATE_STR_TRAIN} | {time}z is missing")
            # try:
            #     filename = [x for x in temp_filenames if time in x and ".idx" not in x][0] #will only be one matching filename @ appropriate time
            #     new_filename = f"urma_{DATE_STR_TRAIN}_t{time}z.nc"
            #     if not os.path.exists(PATH_URMA_TRAIN+f"/{new_filename}"):
            #         t2m = xr.open_dataset(PATH_URMA_TEMP+"/"+filename, engine='cfgrib', decode_timedelta=True)
            #         t2m = t2m.t2m
            #         t2m_subset = t2m.isel(y=slice(IDX_MIN_LAT, IDX_MIN_LAT+IMG_SIZE_LAT),
            #                               x=slice(IDX_MIN_LON, IDX_MIN_LON+IMG_SIZE_LON))
            #         t2m_subset.to_netcdf(PATH_URMA_TRAIN+"/"+new_filename, encoding={"t2m":{"zlib":True, "complevel":9}})
            #         print(f"{new_filename} written to {PATH_URMA_TRAIN}")
            # except:
            #     print(f"{DATE_STR_TRAIN} | {time}z is REALLY missing")

In [ ]:
# All 2024 files to PATH_TEST directory
# for i in range(NUM_DAYS_TEST.days + 1):
for i in range(NUM_DAYS_TEST.days + 1):
    DATE_STR_TEST = date.strftime(START_DATE_TEST + timedelta(days=i), "%Y%m%d")
    filenames = os.listdir(PATH_URMA_ORIGINAL+"/"+DATE_STR_TEST)
    for time in TIME_LIST:
        filename = [x for x in filenames if time in x and ".idx" not in x][0] #will only be one matching filename @ appropriate time
        new_filename = f"urma_{DATE_STR_TEST}_t{time}z.nc"
        if not os.path.exists(PATH_URMA_TEST+f"/{new_filename}"):
            #throws a hissy fit as it can't write an index file on ai-datadepot, but it should still compute fine...
            t2m = xr.open_dataset(PATH_URMA_ORIGINAL+"/"+DATE_STR_TEST+"/"+filename, engine='cfgrib', decode_timedelta=True)
            t2m = t2m.t2m
            t2m_subset = t2m.isel(y=slice(IDX_MIN_LAT, IDX_MIN_LAT+IMG_SIZE_LAT),
                                  x=slice(IDX_MIN_LON, IDX_MIN_LON+IMG_SIZE_LON))
            t2m_subset.to_netcdf(PATH_URMA_TEST+"/"+new_filename)#, encoding={"t2m":{"zlib":True, "complevel":9}})
            print(f"{new_filename} written to {PATH_URMA_TEST}")